In [27]:
# Notebook initialization for consistent paths (repo-aware)
import os, sys, pathlib

# Resolve repo root by walking up until we find 'src'
CWD = pathlib.Path.cwd()
ROOT = CWD
for _ in range(6):
    if (ROOT / 'src').exists():
        break
    ROOT = ROOT.parent
# Fallback to current if not found
if not (ROOT / 'src').exists():
    ROOT = CWD

PROJECT_ROOT = ROOT.resolve()
PROJECT_SRC = PROJECT_ROOT / 'src'
DATA_DIR = PROJECT_SRC / 'data'
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

if str(PROJECT_SRC) not in sys.path:
    sys.path.insert(0, str(PROJECT_SRC))

print(f'PROJECT_ROOT={PROJECT_ROOT}')
print(f'PROJECT_SRC={PROJECT_SRC}')
print(f'RAW_DATA_DIR={RAW_DATA_DIR}')
print(f'PROCESSED_DATA_DIR={PROCESSED_DATA_DIR}')

PROJECT_ROOT=/Users/pierce.bucknerwolfso/Desktop/embeddings_paper/embeddings-service-exploration
PROJECT_SRC=/Users/pierce.bucknerwolfso/Desktop/embeddings_paper/embeddings-service-exploration/src
RAW_DATA_DIR=/Users/pierce.bucknerwolfso/Desktop/embeddings_paper/embeddings-service-exploration/src/data/raw
PROCESSED_DATA_DIR=/Users/pierce.bucknerwolfso/Desktop/embeddings_paper/embeddings-service-exploration/src/data/processed


In [28]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv(f'{PROCESSED_DATA_DIR}/base_loan_data_cleaned.csv')
# Show all columns
pd.set_option('display.max_columns', None)

# Show all rows
pd.set_option('display.max_rows', None)

# Set the maximum column width to display full content
pd.set_option('display.max_colwidth', None)

# Set the maximum sequence items to display full list/array content
pd.set_option('display.max_seq_items', None)


In [30]:
print(df.shape)
df.columns

(1369566, 74)


Index(['loan_amnt', 'term', 'int_rate', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'initial_list_status',
       'collections_12_mths_ex_med', 'application_type', 'tot_coll_amt',
       'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m',
       'total_bal_il', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq',

In [31]:
# strip xx from zip_code
df['zip_code'] = df['zip_code'].str.replace('xx', '')

In [32]:
feature_set = ['addr_state', 'earliest_cr_line', 'emp_length', 'emp_title', 'sub_grade', 'title', 'zip_code', 'avg_cur_bal', 'dti', 'fico_range_high', 'int_rate', 'loan_amnt', 'mort_acc', 'num_op_rev_tl', 'revol_util', 'loan_outcome']
df = df[feature_set]

In [33]:
#find missing values & fix
missing_percentages = (df.isnull().sum() / len(df)) * 100
print(missing_percentages.sort_values(ascending=False))

emp_title           6.423130
emp_length          5.870546
avg_cur_bal         5.132940
num_op_rev_tl       5.131261
mort_acc            3.652982
title               1.237180
revol_util          0.067467
dti                 0.029060
earliest_cr_line    0.002117
zip_code            0.000073
addr_state          0.000000
sub_grade           0.000000
fico_range_high     0.000000
int_rate            0.000000
loan_amnt           0.000000
loan_outcome        0.000000
dtype: float64


In [15]:
def generate_column_summary(df, max_unique_display=5):
    """
    Generates a quick summary for each column in a DataFrame, including
    highest/lowest values and a spread of unique values (up to max_unique_display).

    Args:
        df (pd.DataFrame): The input DataFrame.
        max_unique_display (int): Maximum number of unique values to display.

    Returns:
        str: A formatted string containing the summary for all columns.
    """
    summary_lines = []

    for col in df.columns:
        summary_lines.append(f"--- Column: `{col}` ---")

        # Drop NaN values for summary calculations to avoid issues
        # and then re-add a note about NaN presence later.
        col_data = df[col].dropna()

        if col_data.empty:
            summary_lines.append("  Contains no non-null values.")
            summary_lines.append("")
            continue

        # Check for NaN presence in original column
        if df[col].isnull().any():
            summary_lines.append("  *(Note: Contains missing values)*")

        # Determine data type and get min/max
        if pd.api.types.is_numeric_dtype(col_data):
            summary_lines.append(f"  Type: Numerical")
            summary_lines.append(f"  Lowest Value: {col_data.min():.2f}")
            summary_lines.append(f"  Highest Value: {col_data.max():.2f}")
        elif pd.api.types.is_datetime64_any_dtype(col_data):
            summary_lines.append(f"  Type: Date/Time")
            summary_lines.append(f"  Earliest Date: {col_data.min().strftime('%Y-%m-%d')}")
            summary_lines.append(f"  Latest Date: {col_data.max().strftime('%Y-%m-%d')}")
        else: # Treat as categorical/string
            summary_lines.append(f"  Type: Categorical/String")
            # For strings, min/max are lexicographical
            summary_lines.append(f"  Lexicographically Smallest: '{col_data.min()}'")
            summary_lines.append(f"  Lexicographically Largest: '{col_data.max()}'")

        # Get unique values spread
        unique_values = col_data.unique().tolist()
        num_unique = len(unique_values)

        if num_unique <= max_unique_display:
            # Sort for consistent display
            display_values = sorted(unique_values, key=str) # Use str() for mixed types if needed
            summary_lines.append(f"  Unique Values ({num_unique} total): {display_values}")
        else:
            # Sample unique values if too many
            # Convert to string and sample to avoid issues with non-sortable types
            sampled_values = np.random.choice(unique_values, max_unique_display, replace=False).tolist()
            # Sort the sampled values for consistent output
            display_values = sorted(sampled_values, key=str)
            summary_lines.append(f"  Unique Values ({num_unique} total, showing {max_unique_display} examples): {display_values} ...")

        summary_lines.append("") # Add a blank line for separation

    return "\n".join(summary_lines)

# Generate the summary
summary_output = generate_column_summary(df, max_unique_display=5)

print(summary_output)

--- Column: `addr_state` ---
  Type: Categorical/String
  Lexicographically Smallest: 'AK'
  Lexicographically Largest: 'WY'
  Unique Values (51 total, showing 5 examples): ['CT', 'MA', 'ND', 'NE', 'SC'] ...

--- Column: `earliest_cr_line` ---
  *(Note: Contains missing values)*
  Type: Categorical/String
  Lexicographically Smallest: 'Apr-1934'
  Lexicographically Largest: 'Sep-2015'
  Unique Values (739 total, showing 5 examples): ['Aug-2015', 'Jan-2009', 'Jul-1984', 'Jun-1995', 'Sep-1988'] ...

--- Column: `emp_length` ---
  *(Note: Contains missing values)*
  Type: Categorical/String
  Lexicographically Smallest: '1 year'
  Lexicographically Largest: '< 1 year'
  Unique Values (11 total, showing 5 examples): ['10+ years', '4 years', '7 years', '8 years', '< 1 year'] ...

--- Column: `emp_title` ---
  *(Note: Contains missing values)*
  Type: Categorical/String
  Lexicographically Smallest: '!st Assistant Plumbing Manager'
  Lexicographically Largest: '​License Compliance Investigat

In [16]:
def standardize_string_columns(df):
    """
    Converts all string values in object-type columns to lowercase and removes
    leading/trailing whitespace. NaN values are preserved.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A new DataFrame with string columns standardized.
    """
    df_standardized = df.copy() # Work on a copy to avoid modifying the original DataFrame directly

    for col in df_standardized.columns:
        # Check if the column's dtype is 'object' (which typically indicates strings)
        # and if it contains any non-numeric data that might be string-like.
        if df_standardized[col].dtype == 'object' or pd.api.types.is_string_dtype(df_standardized[col]):
            # Apply .str.lower() and .str.strip() to string values.
            # .str accessor handles NaN values gracefully (they remain NaN).
            df_standardized[col] = df_standardized[col].str.lower().str.strip()
    return df_standardized

# Apply the standardization
df = standardize_string_columns(df)

#df.head()

In [17]:
decimal_precision_map = {
    'loan_amnt': 0,
    'int_rate': 2,
    'annual_inc': 0,
    'dti': 2,
    'fico_range_low': 0,
    'revol_util': 2,
    'pub_rec_bankruptcies': 0,
    'mort_acc': 0,
    'delinq_2yrs': 0,
    'revol_bal': 2,
    'revol_util': 2,
    'total_acc': 0,
    'collections_12_mths_ex_med': 0,
    'tot_coll_amt': 2,
    'total_rev_hi_lim': 2,
    'acc_open_past_24mths': 0,
    'avg_cur_bal': 2,
    'bc_open_to_buy': 2,
    'chargeoff_within_12_mths': 0,
    'delinq_amnt': 2,
    'mo_sin_old_il_acct': 0,
    'mo_sin_old_rev_tl_op': 0,
    'mo_sin_rcnt_rev_tl_op': 0,
    'mo_sin_rcnt_tl': 0,
    'mort_acc': 0,
    'mths_since_recent_bc': 0,
    'mths_since_recent_inq': 0,
    'num_accts_ever_120_pd': 0,
    'num_actv_bc_tl': 0,
    'num_actv_rev_tl': 0,
    'num_bc_sats': 0,
    'num_bc_tl': 0,
    'num_il_tl': 0,
    'num_op_rev_tl': 0,
    'num_rev_accts': 0,
    'num_tl_90g_dpd_24m': 0,
    'num_tl_op_past_12m': 0,
    'pct_tl_nvr_dlq': 1,
    'percent_bc_gt_75': 1,
    'pub_rec_bankruptcies': 0,
    'tax_liens': 0,
    'tot_hi_cred_lim': 2,
    'total_bal_ex_mort': 2,
    'total_bc_limit': 2,
    'inq_last_6mths': 0,
    'open_acc': 0,
    'pub_rec': 0,
}

# Function to format numerical columns and convert to string with padding
def format_numerical_to_string_with_padding(df, precision_map, nan_placeholder="missing_value"):
    """
    Formats specified numerical columns to a given decimal precision, pads the integer part
    with leading zeros based on the maximum integer length in the column, and converts to strings.
    NaN values are filled with a specified placeholder string before formatting.

    Args:
        df (pd.DataFrame): The input DataFrame.
        precision_map (dict): A dictionary mapping column names to desired decimal places (int).
        nan_placeholder (str): The string to replace NaN values with before formatting.

    Returns:
        pd.DataFrame: A new DataFrame with specified numerical columns formatted as strings.
    """
    df_formatted = df.copy()

    # First, calculate max integer length for each column
    max_int_lengths = {}
    for col, precision in precision_map.items():
        if col in df_formatted.columns:
            # Temporarily convert to numeric, coerce errors to NaN, then drop NaNs
            numeric_series = pd.to_numeric(df_formatted[col], errors='coerce').dropna()
            if not numeric_series.empty:
                # Get the absolute value to handle negative numbers for length calculation
                # Convert to string, split by decimal, take integer part, get length
                max_len = numeric_series.apply(
                    lambda x: len(str(int(abs(x)))) if pd.notna(x) else 0
                ).max()
                max_int_lengths[col] = int(max_len)
            else:
                max_int_lengths[col] = 0 # No numeric values to determine length

    for col, precision in precision_map.items():
        if col in df_formatted.columns:
            # Get the determined max integer length for this column, default to 1 if not found
            max_int_len = max_int_lengths.get(col, 1)

            # Apply formatting and padding
            def format_value_with_padding(val):
                if isinstance(val, (float, int)) and not pd.isna(val):
                    # Format to specified decimal places
                    formatted_val = f"{val:.{precision}f}"
                    
                    # Split into integer and decimal parts
                    parts = formatted_val.split('.')
                    integer_part = parts[0]
                    decimal_part = parts[1] if len(parts) > 1 else ''

                    # Handle negative sign for padding
                    sign = ''
                    if integer_part.startswith('-'):
                        sign = '-'
                        integer_part = integer_part[1:]
                    
                    # Pad the integer part with leading zeros
                    padded_integer_part = integer_part.zfill(max_int_len)
                    
                    # Reconstruct the string
                    if precision > 0:
                        return f"{sign}{padded_integer_part}.{decimal_part}"
                    else:
                        return f"{sign}{padded_integer_part}"
                
                # If it's NaN, return the placeholder string
                if pd.isna(val):
                    return nan_placeholder
                
                return str(val) # For any other non-numeric values, convert directly to string

            df_formatted[col] = df_formatted[col].apply(format_value_with_padding)
        else:
            print(f"Warning: Column '{col}' not found in DataFrame for formatting.")
    return df_formatted

# Apply the numerical formatting and string conversion with padding
df = format_numerical_to_string_with_padding(
    df,
    decimal_precision_map,
    nan_placeholder="missing_value" # Consistent string for missing numericals
)

In [18]:
df.head()

,addr_state,earliest_cr_line,emp_length,emp_title,sub_grade,title,zip_code,avg_cur_bal,dti,fico_range_high,int_rate,loan_amnt,mort_acc,num_op_rev_tl,revol_util,loan_outcome
0,pa,aug-2003,10+ years,leadman,c4,debt consolidation,190,020701.00,005.91,679.0,13.99,03600,01,04,029.70,0
1,sd,dec-1999,10+ years,engineer,c1,business,577,009733.00,016.06,719.0,11.99,24700,04,20,019.20,0
2,il,aug-2000,10+ years,truck driver,b4,NaN,605,031617.00,010.78,699.0,10.78,20000,05,04,056.20,0
3,pa,jun-1998,3 years,contract specialist,f1,major purchase,174,027644.00,025.37,699.0,22.45,10400,06,07,064.50,0
4,ga,oct-1987,4 years,veterinary tecnician,c3,debt consolidation,300,002560.00,010.20,694.0,13.44,11950,00,04,068.40,0


In [19]:
# Missing values
missing_percentages = (df.isnull().sum() / len(df)) * 100
print(missing_percentages.sort_values(ascending=False))

emp_title           6.423130
emp_length          5.870546
title               1.237180
earliest_cr_line    0.002117
zip_code            0.000073
addr_state          0.000000
sub_grade           0.000000
avg_cur_bal         0.000000
dti                 0.000000
fico_range_high     0.000000
int_rate            0.000000
loan_amnt           0.000000
mort_acc            0.000000
num_op_rev_tl       0.000000
revol_util          0.000000
loan_outcome        0.000000
dtype: float64


In [ ]:
# Fill NA Values
nan_placeholders = {
    'emp_title': "unknown_job_title",
    'emp_length': "unknown_employment_length",
    'title': "unknown_loan_purpose_title",
    'pub_rec_bankruptcies': "no_record", # Assuming NaN means no record
    'revol_util': "unknown_revolving_utilization",
    # 'dti': "unknown_debt_to_income",
    'earliest_cr_line': "unknown_credit_history_start_date",
    'annual_inc': "unknown_annual_income",
    'mort_acc': "unknown_mortgage_accounts",
    # For columns with 0.000000 missingness, they might not have NaNs,
    # but defining a placeholder is still good practice if NaNs could appear.
    'home_ownership': "unknown_home_ownership_status",
    'loan_amnt': "unknown_loan_amount",
    'term': "unknown_term",
    'verification_status': "unknown_verification_status",
    'issue_d': "unknown_issue_date",
    'addr_state': "unknown_address_state",
    'fico_range_low': "unknown_fico_score",
    'int_rate': "unknown_interest_rate",
    'loan_outcome': "unknown_loan_outcome_status", # Though this might be removed later as leakage
    'collections_12_mths_ex_med': "unknown_collections_12m",
    'tot_coll_amt': "unknown_total_collection_amount",
    'total_rev_hi_lim': "unknown_total_revolving_high_limit",
    'acc_open_past_24mths': "unknown_accounts_opened_24m",
    # 'avg_cur_bal': "unknown_average_current_balance",
    'bc_open_to_buy': "unknown_bankcard_open_to_buy",
    'chargeoff_within_12_mths': "unknown_chargeoffs_12m",
    'delinq_amnt': "unknown_delinquent_amount",
    'mo_sin_old_il_acct': "unknown_months_since_old_il_account",
    'mo_sin_old_rev_tl_op': "unknown_months_since_old_revolving_account",
    'mo_sin_rcnt_rev_tl_op': "unknown_months_since_recent_revolving_account",
    'mo_sin_rcnt_tl': "unknown_months_since_recent_account",
    'mort_acc': "unknown_mortgage_accounts",
    'mths_since_recent_bc': "unknown_months_since_recent_bankcard",
    'mths_since_recent_inq': "unknown_months_since_recent_inquiry",
    'num_accts_ever_120_pd': "unknown_accounts_ever_120_pd",
    'num_actv_bc_tl': "unknown_active_bankcard_accounts",
    'num_actv_rev_tl': "unknown_active_revolving_accounts",
    'num_bc_sats': "unknown_satisfactory_bankcard_accounts",
    'num_bc_tl': "unknown_bankcard_accounts",
    'num_il_tl': "unknown_installment_accounts",
    'num_op_rev_tl': "unknown_open_revolving_accounts",
    'num_rev_accts': "unknown_revolving_accounts",
    'num_tl_90g_dpd_24m': "unknown_90_dpd_24m",
    'num_tl_op_past_12m': "unknown_accounts_opened_12m",
    'pct_tl_nvr_dlq': "unknown_percent_never_delinquent",
    'percent_bc_gt_75': "unknown_percent_bc_gt_75",
    'tax_liens': "unknown_tax_liens",
    'tot_hi_cred_lim': "unknown_total_high_credit_limit",
    'total_bal_ex_mort': "unknown_total_balance_ex_mortgage",
    'total_bc_limit': "unknown_total_bankcard_limit",
    'inq_last_6mths': "unknown_inquiries_6m",
    'open_acc': "unknown_open_accounts",
    'pub_rec': "unknown_public_records",
    'tot_cur_bal': "unknown_total_current_balance",
    'bc_util': "unknown_bankcard_utilization",
    'num_rev_tl_bal_gt_0': "unknown_revolving_accounts_with_balance",
    'num_sats': "unknown_satisfactory_bankcard_accounts",
    'total_il_high_credit_limit': "unknown_total_installment_high_credit_limit",
    
}

# --- Function to fill NaNs with domain-specific placeholders ---
def fill_nans_with_domain_specific_placeholders(df, placeholders_map):
    """
    Fills NaN values in specified columns of a DataFrame with domain-specific string placeholders.

    Args:
        df (pd.DataFrame): The input DataFrame.
        placeholders_map (dict): A dictionary mapping column names to their NaN placeholder strings.

    Returns:
        pd.DataFrame: A new DataFrame with NaNs filled according to the map.
    """
    df_filled = df.copy()

    for col, placeholder in placeholders_map.items():
        if col in df_filled.columns:
            # Ensure the column is of a type that can hold strings, or convert it
            # if it's purely numeric and will be converted to string later.
            # Using .astype(str) before fillna can be an option for consistency,
            # but fillna itself will often coerce to object dtype if needed.
            df_filled[col] = df_filled[col].fillna(placeholder)
        else:
            print(f"Warning: Column '{col}' not found in DataFrame for NaN filling.")
    return df_filled

# Apply the NaN filling
df = fill_nans_with_domain_specific_placeholders(df, nan_placeholders)


In [21]:
# Check for NaNs in the data
df.isnull().sum().sort_values(ascending=False)

zip_code            1
addr_state          0
earliest_cr_line    0
emp_length          0
emp_title           0
sub_grade           0
title               0
avg_cur_bal         0
dti                 0
fico_range_high     0
int_rate            0
loan_amnt           0
mort_acc            0
num_op_rev_tl       0
revol_util          0
loan_outcome        0
dtype: int64

In [22]:
df.dropna(inplace=True)
#df_cleaned.head()

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1369565 entries, 0 to 1369565
Data columns (total 16 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   addr_state        1369565 non-null  object 
 1   earliest_cr_line  1369565 non-null  object 
 2   emp_length        1369565 non-null  object 
 3   emp_title         1369565 non-null  object 
 4   sub_grade         1369565 non-null  object 
 5   title             1369565 non-null  object 
 6   zip_code          1369565 non-null  object 
 7   avg_cur_bal       1369565 non-null  object 
 8   dti               1369565 non-null  object 
 9   fico_range_high   1369565 non-null  float64
 10  int_rate          1369565 non-null  object 
 11  loan_amnt         1369565 non-null  object 
 12  mort_acc          1369565 non-null  object 
 13  num_op_rev_tl     1369565 non-null  object 
 14  revol_util        1369565 non-null  object 
 15  loan_outcome      1369565 non-null  int64  
dtypes: fl

In [24]:
# Now, check the new value counts
print(df['loan_outcome'].value_counts()/len(df))

loan_outcome
0    0.78765
1    0.21235
Name: count, dtype: float64


In [26]:
# Save df for embeddings
df.to_csv(f'{PROCESSED_DATA_DIR}/final_features_for_embeddings.csv', index=False)